# [第7章: データベース](http://www.cl.ecei.tohoku.ac.jp/nlp100/#ch7)
[artist.json.gz](http://www.cl.ecei.tohoku.ac.jp/nlp100/data/artist.json.gz)は，オープンな音楽データベース[MusicBrainz](https://musicbrainz.org/)の中で，アーティストに関するものをJSON形式に変換し，gzip形式で圧縮したファイルである．このファイルには，1アーティストに関する情報が1行にJSON形式で格納されている．JSON形式の概要は以下の通りである．

| フィールド | 型 | 内容 | 例 |
| :--------- | :- | :--- | :- |
| id | ユニーク識別子 | 整数 | 20660 |
| gid | グローバル識別子 | 文字列 | "ecf9f3a3-35e9-4c58-acaa-e707fba45060" |
| name | アーティスト名 | 文字列 | "Oasis" |
| sort_name | アーティスト名（辞書順整列用） | 文字列 | "Oasis" |
| area | 活動場所 | 文字列 | "United Kingdom" |
| aliases | 別名 | 辞書オブジェクトのリスト |  |
| aliases[].name | 別名 | 文字列 | "オアシス" |
| aliases[].sort_name | 別名（整列用） | 文字列 | "オアシス" |
| begin | 活動開始日 | 辞書 |  |
| begin.year | 活動開始年 | 整数 | 1991 |
| begin.month | 活動開始月 | 整数 |  |
| begin.date | 活動開始日 | 整数 |  |
| end | 活動終了日 | 辞書 |  |
| end.year | 活動終了年 | 整数 | 2009 |
| end.month | 活動終了月 | 整数 | 8 |
| end.date | 活動終了日 | 整数 | 28 |
| tags | タグ | 辞書オブジェクトのリスト |  |
| tags[].count | タグ付けされた回数 | 整数 | 1 |
| tags[].value | タグ内容 | 文字列 | "rock" |
| rating | レーティング | 辞書オブジェクト |  |
| rating.count | レーティングの投票数 | 整数 | 13 |
| rating.value | レーティングの値（平均値） | 整数 | 86 |

artist.json.gzのデータをKey-Value-Store (KVS) およびドキュメント志向型データベースに格納・検索することを考える．KVSとしては，[LevelDB](http://leveldb.org/)，[Redis](http://redis.io/)，[KyotoCabinet](http://fallabs.com/kyotocabinet/)等を用いよ．ドキュメント志向型データベースとして，[MongoDB](http://www.mongodb.org/)を採用したが，[CouchDB](http://couchdb.apache.org/)や[RethinkDB](http://rethinkdb.com/)等を用いてもよい．

In [1]:
!ls ../Corpus/artist.json.gz

../Corpus/artist.json.gz


## 60. KVSの構築
Key-Value-Store (KVS) を用い，アーティスト名（name）から活動場所（area）を検索するためのデータベースを構築せよ．

In [2]:
import gzip
import json
import plyvel

In [3]:
db_60 = plyvel.DB("artist_60", create_if_missing=True)

In [4]:
%%time
with gzip.open("../Corpus/artist.json.gz", mode="rt") as artists_json, db_60.write_batch() as wb_60:
    for artist_json in artists_json:
        artist = json.loads(artist_json)
        if "name" not in artist or "area" not in artist:
            continue
        k = artist["name"]
        v = artist["area"]
        wb_60.put(k.encode("utf-8"), v.encode("utf-8"))

CPU times: user 7.83 s, sys: 216 ms, total: 8.05 s
Wall time: 8.21 s


In [5]:
db_60.close()

## 61. KVSの検索
60で構築したデータベースを用い，特定の（指定された）アーティストの活動場所を取得せよ．

In [6]:
db_60 = plyvel.DB("artist_60")

In [7]:
db_60.get(b"Oasis")

b'United Kingdom'

In [8]:
db_60.close()

## 62. KVS内の反復処理
60で構築したデータベースを用い，活動場所が「Japan」となっているアーティスト数を求めよ．

In [9]:
db_60 = plyvel.DB("artist_60")

In [10]:
count = 0
for _, v in db_60:
    if v == b"Japan":
        count += 1
count

22128

In [11]:
db_60.close()

## 63. オブジェクトを値に格納したKVS
KVSを用い，アーティスト名（name）からタグと被タグ数（タグ付けされた回数）のリストを検索するためのデータベースを構築せよ．さらに，ここで構築したデータベースを用い，アーティスト名からタグと被タグ数を検索せよ．

| フィールド | 型 | 内容 | 例 |
| :--------- | :- | :--- | :- |
| name | アーティスト名 | 文字列 | "Oasis" |
| tags | タグ | 辞書オブジェクトのリスト |  |
| tags[].count | タグ付けされた回数 | 整数 | 1 |
| tags[].value | タグ内容 | 文字列 | "rock" |

In [12]:
db_63 = plyvel.DB("artist_63", create_if_missing=True)

In [13]:
%%time
with gzip.open("../Corpus/artist.json.gz", mode="rt") as artists_json, db_63.write_batch() as wb_63:
    i = 0
    for artist_json in artists_json:
        artist = json.loads(artist_json)
        if "name" not in artist or "tags" not in artist:
            continue
        k = artist["name"]
        v = ""
        for tag in artist["tags"]:
            v += "%s=%s, " % (tag["value"], tag["count"])
        wb_63.put(k.encode("utf-8"), v[:-2].encode("utf-8"))

CPU times: user 7.3 s, sys: 27.5 ms, total: 7.32 s
Wall time: 7.37 s


In [14]:
db_63.get(b"Rezerwat")

b'polish rock=1, new wave=1'

In [15]:
db_63.close()

## 64. MongoDBの構築
アーティスト情報（artist.json.gz）をデータベースに登録せよ．さらに，次のフィールドでインデックスを作成せよ: name, aliases.name, tags.value, rating.value

In [56]:
import pymongo

In [62]:
db_mongo = pymongo.MongoClient().chapter7
artists_mongo = db_mongo.artists

In [63]:
%%time
with gzip.open("../Corpus/artist.json.gz", mode="rt") as artists_json:
    artists = [json.loads(artist_json) for artist_json in artists_json]
    artists_mongo.insert_many(artists)

CPU times: user 24.1 s, sys: 401 ms, total: 24.5 s
Wall time: 29.2 s


In [64]:
%%time
index_64 = [("name", pymongo.ASCENDING),
            ("aliases[].name", pymongo.ASCENDING),
            ("tags[].value", pymongo.ASCENDING),
            ("rating.value", pymongo.ASCENDING)]
artists_mongo.create_index(index_64)
artists_mongo.index_information()

CPU times: user 3.86 ms, sys: 206 µs, total: 4.06 ms
Wall time: 4.71 s


{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'chapter7.artists'},
 'name_1_aliases[].name_1_tags[].value_1_rating.value_1': {'v': 2,
  'key': [('name', 1),
   ('aliases[].name', 1),
   ('tags[].value', 1),
   ('rating.value', 1)],
  'ns': 'chapter7.artists'}}

## 65. MongoDBの検索
MongoDBのインタラクティブシェルを用いて，"Queen"というアーティストに関する情報を取得せよ．さらに，これと同様の処理を行うプログラムを実装せよ．

In [65]:
for i, artists_find in enumerate(artists_mongo.find({"name": "Queen"})):
    print("[%d/%d]" % (i+1, artists_mongo.count_documents({"name": "Queen"})))
    for k, v in artists_find.items():
        print("\t%s\t%s" % (k, v))
    print()

[1/3]
	_id	5e8f3a66d11b16273bde86d0
	name	Queen
	area	Japan
	gender	Female
	tags	[{'count': 1, 'value': 'kamen rider w'}, {'count': 1, 'value': 'related-akb48'}]
	sort_name	Queen
	ended	True
	gid	420ca290-76c5-41af-999e-564d7c71f1a7
	type	Character
	id	701492
	aliases	[{'name': 'Queen', 'sort_name': 'Queen'}]

[2/3]
	_id	5e8f3a68d11b16273be107d4
	ended	True
	gid	5eecaf18-02ec-47af-a4f2-7831db373419
	sort_name	Queen
	id	992994
	name	Queen

[3/3]
	_id	5e8f3a66d11b16273bdf4d7c
	rating	{'count': 24, 'value': 92}
	begin	{'date': 27, 'month': 6, 'year': 1970}
	name	Queen
	area	United Kingdom
	tags	[{'count': 2, 'value': 'hard rock'}, {'count': 1, 'value': '70s'}, {'count': 1, 'value': 'queen family'}, {'count': 1, 'value': '90s'}, {'count': 1, 'value': '80s'}, {'count': 1, 'value': 'glam rock'}, {'count': 4, 'value': 'british'}, {'count': 1, 'value': 'english'}, {'count': 2, 'value': 'uk'}, {'count': 1, 'value': 'pop/rock'}, {'count': 1, 'value': 'pop-rock'}, {'count': 1, 'value': 'britanniq

## 66. 検索件数の取得
MongoDBのインタラクティブシェルを用いて，活動場所が「Japan」となっているアーティスト数を求めよ．

In [66]:
artists_mongo.count_documents({"area": "Japan"})

22821

## 67. 複数のドキュメントの取得
特定の（指定した）別名を持つアーティストを検索せよ．

In [67]:
def print_aliased_name(name: str):
    for i, artists_find in enumerate(artists_mongo.find({"aliases.name": name})):
        print("[%d/%d]" % (i+1, artists_mongo.count_documents({"aliases.name": name})))
        for k, v in artists_find.items():
            print("\t%s\t%s" % (k, v))
        print()

In [68]:
print_aliased_name("オアシス")

[1/1]
	_id	5e8f3a66d11b16273bdf728f
	rating	{'count': 13, 'value': 86}
	begin	{'year': 1991}
	end	{'date': 28, 'month': 8, 'year': 2009}
	name	Oasis
	area	United Kingdom
	tags	[{'count': 1, 'value': 'rock'}, {'count': 3, 'value': 'britpop'}, {'count': 4, 'value': 'british'}, {'count': 1, 'value': 'uk'}, {'count': 1, 'value': 'britannique'}, {'count': 1, 'value': 'rock and indie'}, {'count': 1, 'value': 'england'}, {'count': 1, 'value': 'manchester'}]
	sort_name	Oasis
	ended	True
	gid	39ab1aed-75e0-4140-bd47-540276886b60
	type	Group
	id	20660
	aliases	[{'name': 'OASIS', 'sort_name': 'OASIS'}, {'name': 'オアシス', 'sort_name': 'オアシス'}]



## 68. ソート
"dance"というタグを付与されたアーティストの中でレーティングの投票数が多いアーティスト・トップ10を求めよ．

In [72]:
filter_68 = {"tags.value": "dance", "rating": {"$exists": "true"}}
sort_68 = [("rating.count", pymongo.DESCENDING)]
cursor_68 = artists_mongo.find(filter_68,
                               limit=10,
                               sort=sort_68)
for i, artists_find in enumerate(cursor_68):
    print("[%d/10]" % (i+1))
    print("\tname\t%s" % artists_find["name"])
    print("\trating.count\t%s" % artists_find["rating"]["count"])
    print()

[1/10]
	name	Madonna
	rating.count	26

[2/10]
	name	Björk
	rating.count	23

[3/10]
	name	The Prodigy
	rating.count	23

[4/10]
	name	Rihanna
	rating.count	15

[5/10]
	name	Britney Spears
	rating.count	13

[6/10]
	name	Maroon 5
	rating.count	11

[7/10]
	name	Adam Lambert
	rating.count	7

[8/10]
	name	Fatboy Slim
	rating.count	7

[9/10]
	name	Basement Jaxx
	rating.count	6

[10/10]
	name	Cornershop
	rating.count	5



## 69. Webアプリケーションの作成
ユーザから入力された検索条件に合致するアーティストの情報を表示するWebアプリケーションを作成せよ．アーティスト名，アーティストの別名，タグ等で検索条件を指定し，アーティスト情報のリストをレーティングの高い順などで整列して表示せよ．